In [4]:
pip install pyEDFlib

Note: you may need to restart the kernel to use updated packages.


In [5]:
# libraries installation
from pyedflib import EdfReader
from numpy import zeros, arange
# import csv
import pandas as pd
from csv import reader, writer
import os

**PATIENT-NON-SPECIFIC and PS-Inter Specific**


In [ ]:
#paths defines
#create these directories if not existing
Edfpath='final_data/'
Datasetpath='Main Dataset/'
tempath='chb_csv/'

# read the excel file made for patient data
excelData=pd.read_csv(r'CHB-MIT DB timestamp.csv', header=None)
total_no_of_edfs=591
excelData

,0,1,2,3,4,5,6,7,8,9,...,25,26,27,28,29,30,31,32,33,34
0,Patient,Sub File,No of channels,Channels not used,No of Seizure,Seizure Start Time,Seizure End Time,Duration,Sampling Frequency,Ictal Start Row,...,NS_Ictal Start Time,NS_Ictal End Time,NS_Ictal Start Row,NS_Ictal End Row,Non Seizure SubFile2,Peri Ictal Duration,NS_Peri Ictal Start Time,NS_Peri Ictal End Time,NS_Peri-Ictal Start Row,NS_Peri Ictal End Row
1,1,chb01_03,23,NaN,1,2996,3036,40,256,766976,...,1780,1820,455680,465920,chb01_02,60,1770,1830,453120,468480
2,1,chb01_04,23,NaN,1,1467,1494,27,256,375552,...,1786,1813,457216,464128,chb01_06,47,1776,1823,454656,466688
3,1,chb01_15,23,NaN,1,1732,1772,40,256,443392,...,1780,1820,455680,465920,chb01_08,60,1770,1830,453120,468480
4,1,chb01_16,23,NaN,1,1015,1066,51,256,259840,...,1775,1826,454400,467456,chb01_10,71,1765,1836,451840,470016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193,24,chb24_13,23,NaN,1,3288,3304,16,256,841728,...,500,516,128000,132096,chb02_18,36,1774,1810,454144,463360
194,24,chb24_14,23,NaN,1,1939,1966,27,256,496384,...,500,527,128000,134912,chb02_21,47,1774,1821,454144,466176
195,24,chb24_15,23,NaN,1,3552,3569,17,256,909312,...,500,517,128000,132352,chb02_22,37,1774,1811,454144,463616
196,24,chb24_17,23,NaN,1,3515,3581,66,256,899840,...,500,566,128000,144896,chb02_23,86,1774,1860,454144,476160


**ICTAL**

In [ ]:
seizure_no=0
for i in range(1,total_no_of_edfs):

  edfFile=excelData[1][i]
  patientno= excelData[0][i]
  subfile=excelData[1][i]

  #read Ictal start and end rows
  IctalS= int(excelData[9][i])
  IctalE= int(excelData[10][i])

  #read edf file
  f=EdfReader(Edfpath+ edfFile+'.edf')
  n = f.signals_in_file
  signal_labels = f.getSignalLabels()
  print(signal_labels)
  data = zeros((n,f.getNSamples()[0]))
  data
  for k in arange(n):
      data[k, :] = f.readSignal(k)

  #store all data
  data_eeg = data

  #close the edf file
  f._close()
  del f
  print(data_eeg.shape)

  df = pd.DataFrame(data_eeg)
  total_cols= len(df.columns)
  print(total_cols)

  #cut the data for ictal
  cols=list(range(0,IctalS))
  df.drop(columns=cols, axis=1,inplace=True)
  cols=list(range(IctalE+1,total_cols))
  df.drop(columns=cols, axis=1,inplace=True)
  print(df.shape)

  # add class label
  df[len(df.columns)]=1

  #save the updated csv file
  df.to_csv(tempath+'temp2.csv', index=False)


  # create category specific folder
  newpath = 'Main Dataset/Patient Non specific/Seizure Patterns/Ictal/'
  if not os.path.exists(newpath):
    os.makedirs(newpath)

  # create pateint specific folder
  folderpath = 'Main Dataset/Patient Specific/Inter-specific/Seizure Patterns/chb_' + str(patientno)+'/'
  if not os.path.exists(folderpath):
    os.makedirs(folderpath)

  # create category specific folder
  newpath2= folderpath + 'Ictal/'
  if not os.path.exists(newpath2):
    os.makedirs(newpath2)

  # intraspecific
  folderpath2 = 'Main Dataset/Patient Specific/Intra-specific/Seizure Patterns/chb_' + str(patientno)+'/'
  if not os.path.exists(folderpath):
    os.makedirs(folderpath)

  # create subfile specific folder
  newpath3 = folderpath2 +subfile+ '/'
  if not os.path.exists(newpath3 ):
    os.makedirs(newpath3)

  # transpose the csv
  with open(tempath+'temp2.csv') as f, open(newpath + str(i+seizure_no) +'.csv', 'w') as fw:
      writer(fw, delimiter=',').writerows(zip(*reader(f, delimiter=',')))

  # read and drop the first col
  df=pd.read_csv(newpath + str(i+seizure_no) +'.csv', header=None)
  df.drop(columns=0, axis=1,inplace=True)

  # Non Specific
  df.to_csv(newpath +str(i+seizure_no) +'.csv', index=False, header=False)

  # InterSpecific

  if patientno!='23' and patientno!='24':
    df.to_csv(newpath2 + subfile +'_ictal.csv', index=False, header=False)

  # Intraspecific
  if patientno=='1' or patientno=='21' :
    df.to_csv(newpath3 + subfile +'_ictal.csv', index=False, header=False)

**POST-ICTAL**

In [ ]:
seizure_no=0
for i in range(1,total_no_of_edfs):

  edfFile=excelData[1][i]
  patientno= excelData[0][i]
  subfile=excelData[1][i]
# read PreIctal start and end rows
  Start= int(excelData[13][i])
  End= int(excelData[14][i])

# read edf file
  f=EdfReader(Edfpath+ edfFile+'.edf')
  n = f.signals_in_file
  signal_labels = f.getSignalLabels()
  print(signal_labels)
  data = zeros((n,f.getNSamples()[0]))
  data
  for k in arange(n):
      data[k, :] = f.readSignal(k)

 #store all data
  data_eeg = data


  #close the edf file
  f._close()
  del f
  print(data_eeg.shape)

  df = pd.DataFrame(data_eeg)
  total_cols= len(df.columns)
  print(total_cols)

  #cut the data for postictal
  cols=list(range(0,Start))
  df.drop(columns=cols, axis=1,inplace=True)
  cols=list(range(End+1,total_cols))
  df.drop(columns=cols, axis=1,inplace=True)
  print(df.shape)

  # add class label
  df[len(df.columns)]=3

  #save the updated csv file
  df.to_csv(tempath+'temp2.csv', index=False)


  # create category specific folder
  newpath = 'Main Dataset/Patient Non specific/Seizure Patterns/Post-Ictal/'
  if not os.path.exists(newpath):
    os.makedirs(newpath)

  # create pateint specific folder
  folderpath = 'Main Dataset/Patient Specific/Inter-specific/Seizure Patterns/chb_' + str(patientno)+'/'
  if not os.path.exists(folderpath):
    os.makedirs(folderpath)

  # create category specific folder
  newpath2= folderpath + 'Post-Ictal/'
  if not os.path.exists(newpath2):
    os.makedirs(newpath2)

  #intraspecific
  folderpath2 = 'Main Dataset/Patient Specific/Intra-specific/Seizure Patterns/chb_' + str(patientno)+'/'
  if not os.path.exists(folderpath):
    os.makedirs(folderpath)

  # create subfile specific folder
  newpath3 = folderpath2 +subfile+ '/'
  if not os.path.exists(newpath3 ):
    os.makedirs(newpath3)

  # transpose the csv
  with open(tempath+'temp2.csv') as f, open(newpath + str(i+seizure_no) +'.csv', 'w') as fw:
      writer(fw, delimiter=',').writerows(zip(*reader(f, delimiter=',')))

  # read and drop the first col
  df=pd.read_csv(newpath + str(i+seizure_no) +'.csv', header=None)
  df.drop(columns=0, axis=1,inplace=True)

  # Non Specific
  df.to_csv(newpath +str(i+seizure_no) +'.csv', index=False, header=False)

  # InterSpecific
  if patientno!='23' and patientno!='24' :
    df.to_csv(newpath2 + subfile +'_post-ictal.csv', index=False, header=False)

  # Intraspecific
  if patientno=='1' :
    df.to_csv(newpath3 + subfile +'_post-ictal.csv', index=False, header=False)
  if patientno=='21':
    df.to_csv(newpath3 + subfile +'_post-ictal.csv', index=False, header=False)

**PRE_ICTAL**

In [ ]:
seizure_no=0
for i in range(1,total_no_of_edfs):

  edfFile=excelData[1][i]
  patientno= excelData[0][i]
  subfile=excelData[1][i]
# read PreIctal start and end rows
  Start= int(excelData[17][i])
  End= int(excelData[18][i])

# read edf file
  f=EdfReader(Edfpath+ edfFile+'.edf')
  n = f.signals_in_file
  signal_labels = f.getSignalLabels()
  print(signal_labels)
  data = zeros((n,f.getNSamples()[0]))
  data
  for k in arange(n):
      data[k, :] = f.readSignal(k)

  #store all data
  data_eeg = data


  #close the edf file
  f._close()
  del f
  print(data_eeg.shape)

  df = pd.DataFrame(data_eeg)

  total_cols= len(df.columns)
  print(total_cols)

  #cut the data for preictal
  cols=list(range(0,Start))
  df.drop(columns=cols, axis=1,inplace=True)
  cols=list(range(End+1,total_cols))
  df.drop(columns=cols, axis=1,inplace=True)
  print(df.shape)

  # add class label
  df[len(df.columns)]=2

  #save the updated csv file
  df.to_csv(tempath+'temp2.csv', index=False)


  # create category specific folder
  newpath = 'Main Dataset/Patient Non specific/Seizure Patterns/Pre-Ictal/'
  if not os.path.exists(newpath):
    os.makedirs(newpath)

  # create pateint specific folder
  folderpath = 'Main Dataset/Patient Specific/Inter-specific/Seizure Patterns/chb_' + str(patientno)+'/'
  if not os.path.exists(folderpath):
    os.makedirs(folderpath)

  # create category specific folder
  newpath2= folderpath + 'Pre-Ictal/'
  if not os.path.exists(newpath2):
    os.makedirs(newpath2)

  # intraspecific
  folderpath2 = 'Main Dataset/Patient Specific/Intra-specific/Seizure Patterns/chb_' + str(patientno)+'/'
  if not os.path.exists(folderpath):
    os.makedirs(folderpath)

  # create subfile specific folder
  newpath3 = folderpath2 +subfile+ '/'
  if not os.path.exists(newpath3 ):
    os.makedirs(newpath3)

  # transpose the csv
  with open(tempath+'temp2.csv') as f, open(newpath + str(i+seizure_no) +'.csv', 'w') as fw:
      writer(fw, delimiter=',').writerows(zip(*reader(f, delimiter=',')))

  # read and drop the first col
  df=pd.read_csv(newpath + str(i+seizure_no) +'.csv', header=None)
  df.drop(columns=0, axis=1,inplace=True)

  # Non Specific
  df.to_csv(newpath +str(i+seizure_no) +'.csv', index=False, header=False)

  # InterSpecific
  if patientno!='23' and patientno!='24' :
    df.to_csv(newpath2 + subfile +'_pre-ictal.csv', index=False, header=False)

  # Intraspecific
  if patientno=='1' or patientno=='21' :
    df.to_csv(newpath3 + subfile +'_pre-ictal.csv', index=False, header=False)

**PERI_ICTAL**

In [ ]:

seizure_no=0
for i in range(1,total_no_of_edfs):

  edfFile=excelData[1][i]
  patientno= excelData[0][i]
  subfile=excelData[1][i]
# read PreIctal start and end rows
  Start= int(excelData[21][i])
  End= int(excelData[22][i])

# read edf file
  f=EdfReader(Edfpath+ edfFile+'.edf')
  n = f.signals_in_file
  signal_labels = f.getSignalLabels()
  print(signal_labels)
  data = zeros((n,f.getNSamples()[0]))
  data
  for k in arange(n):
      data[k, :] = f.readSignal(k)

 #store all data
  data_eeg = data


  #close the edf file
  f._close()
  del f
  print(data_eeg.shape)

  df = pd.DataFrame(data_eeg)
  total_cols= len(df.columns)
  print(total_cols)

  #cut the data for periictal
  cols=list(range(0,Start))
  df.drop(columns=cols, axis=1,inplace=True)
  cols=list(range(End+1,total_cols))
  df.drop(columns=cols, axis=1,inplace=True)
  print(df.shape)

  # add class label
  df[len(df.columns)]=4

  #save the updated csv file
  df.to_csv(tempath+'temp2.csv', index=False)


  # create category specific folder
  newpath = 'Main Dataset/Patient Non specific/Seizure Patterns/Peri-Ictal/'
  if not os.path.exists(newpath):
    os.makedirs(newpath)

  # create pateint specific folder
  folderpath = 'Main Dataset/Patient Specific/Inter-specific/Seizure Patterns/chb_' + str(patientno)+'/'
  if not os.path.exists(folderpath):
    os.makedirs(folderpath)

  # create category specific folder
  newpath2= folderpath + 'Peri-Ictal/'
  if not os.path.exists(newpath2):
    os.makedirs(newpath2)

  # intraspecific
  folderpath2 = 'Main Dataset/Patient Specific/Intra-specific/Seizure Patterns/chb_' + str(patientno)+'/'
  if not os.path.exists(folderpath):
    os.makedirs(folderpath)

  # create subfile specific folder
  newpath3 = folderpath2 +subfile+ '/'
  if not os.path.exists(newpath3 ):
    os.makedirs(newpath3)

  # transpose the csv
  with open(tempath+'temp2.csv') as f, open(newpath + str(i+seizure_no) +'.csv', 'w') as fw:
      writer(fw, delimiter=',').writerows(zip(*reader(f, delimiter=',')))

  # read and drop the first col
  df=pd.read_csv(newpath + str(i+seizure_no) +'.csv', header=None)
  df.drop(columns=0, axis=1,inplace=True)

  # Non Specific
  df.to_csv(newpath +str(i+seizure_no) +'.csv', index=False, header=False)

  # InterSpecific
  if patientno!='23' and patientno!='24' :
    df.to_csv(newpath2 + subfile +'_peri-ictal.csv', index=False, header=False)

  # Intraspecific
  if patientno=='1' or patientno=='21' :
    df.to_csv(newpath3 + subfile +'_peri-ictal.csv', index=False, header=False)

**NON-SEIZURE- ICTAL**

In [ ]:
seizure_no=0
for i in range(1,total_no_of_edfs):

  edfFile=excelData[23][i]
  patientno= excelData[0][i]
  subfile=excelData[23][i]

# read start and end rows
  Start= int(excelData[27][i])
  End= int(excelData[28][i])

# read edf file
  f=EdfReader(Edfpath+ edfFile+'.edf')
  n = f.signals_in_file
  signal_labels = f.getSignalLabels()
  print(signal_labels)
  data = zeros((n,f.getNSamples()[0]))
  data
  for k in arange(n):
      data[k, :] = f.readSignal(k)

 #store all data
  data_eeg = data


  #close the edf file
  f._close()
  del f
  print(data_eeg.shape)

  df = pd.DataFrame(data_eeg)
  total_cols= len(df.columns)
  print(total_cols)

  #cut the data for non seizure ictal
  cols=list(range(0,Start))
  df.drop(columns=cols, axis=1,inplace=True)
  cols=list(range(End+1,total_cols))
  df.drop(columns=cols, axis=1,inplace=True)
  print(df.shape)

  # add class label
  df[len(df.columns)]=5

  #save the updated csv file
  df.to_csv(tempath+'temp2.csv', index=False)

  # create category specific folder
  newpath = 'Main Dataset/Patient Non specific/Non-Seizure Patterns/Non-Seizure(Ictal)/'
  if not os.path.exists(newpath):
    os.makedirs(newpath)

  # create pateint specific folder
  folderpath = 'Main Dataset/Patient Specific/Inter-specific/Non-Seizure Patterns/chb_' + str(patientno)+'/'
  if not os.path.exists(folderpath):
    os.makedirs(folderpath)

  # create category specific folder
  newpath2= folderpath + 'Non-Seizure(Ictal)/'
  if not os.path.exists(newpath2):
    os.makedirs(newpath2)

  #Intraspecific
  folderpath2 = 'Main Dataset/Patient Specific/Intra-specific/Non-Seizure Patterns/chb_' + str(patientno)+'/'
  if not os.path.exists(folderpath):
    os.makedirs(folderpath)

  # create subfile specific folder
  newpath3 = folderpath2 +subfile+ '/'
  if not os.path.exists(newpath3 ):
    os.makedirs(newpath3)

  # transpose the csv
  with open(tempath+'temp2.csv') as f, open(newpath + str(i+seizure_no) +'.csv', 'w') as fw:
      writer(fw, delimiter=',').writerows(zip(*reader(f, delimiter=',')))

  # read and drop the first col
  df=pd.read_csv(newpath + str(i+seizure_no) +'.csv', header=None)
  df.drop(columns=0, axis=1,inplace=True)

  # Non Specific
  df.to_csv(newpath +str(i+seizure_no) +'.csv', index=False, header=False)

  # InterSpecific
  if patientno!='23' and patientno!='24' :
    df.to_csv(newpath2 + subfile +'_non-seizure(ictal).csv', index=False, header=False)

  # Intraspecific
  if patientno=='1' or patientno=='21' :
    df.to_csv(newpath3 + subfile +'_non-seizure(ictal).csv', index=False, header=False)

**NON-SEIZURE- PERI-ICTAL**

In [ ]:
seizure_no=0
for i in range(1,total_no_of_edfs):

  edfFile=excelData[29][i]
  patientno= excelData[0][i]
  subfile=excelData[29][i]

# read start and end rows
  Start= int(excelData[33][i])
  End= int(excelData[34][i])


# read edf file
  f=EdfReader(Edfpath+ edfFile+'.edf')
  n = f.signals_in_file
  signal_labels = f.getSignalLabels()
  print(signal_labels)
  data = zeros((n,f.getNSamples()[0]))
  data
  for k in arange(n):
      data[k, :] = f.readSignal(k)

 #store all data
  data_eeg = data


  #close the edf file
  f._close()
  del f
  print(data_eeg.shape)

  df = pd.DataFrame(data_eeg)
  total_cols= len(df.columns)
  print(total_cols)

  #cut the data for non seizure ictal
  cols=list(range(0,Start))
  df.drop(columns=cols, axis=1,inplace=True)
  cols=list(range(End+1,total_cols))
  df.drop(columns=cols, axis=1,inplace=True)
  print(df.shape)

  # add class label
  df[len(df.columns)]=6

  #save the updated csv file
  df.to_csv(tempath+'temp2.csv', index=False)

  # create category specific folder
  newpath = 'Main Dataset/Patient Non specific/Non-Seizure Patterns/Non-Seizure(Peri-Ictal)/'
  if not os.path.exists(newpath):
    os.makedirs(newpath)

  # create pateint specific folder
  folderpath = 'Main Dataset/Patient Specific/Inter-specific/Non-Seizure Patterns/chb_' + str(patientno)+'/'
  if not os.path.exists(folderpath):
    os.makedirs(folderpath)

  # create category specific folder
  newpath2= folderpath + 'Non-Seizure(Peri-Ictal)/'
  if not os.path.exists(newpath2):
    os.makedirs(newpath2)

  # intraspecific
  folderpath2 = 'Main Dataset/Patient Specific/Intra-specific/Non-Seizure Patterns/chb_' + str(patientno)+'/'
  if not os.path.exists(folderpath):
    os.makedirs(folderpath)

  # create subfile specific folder
  newpath3 = folderpath2 +subfile+ '/'
  if not os.path.exists(newpath3 ):
    os.makedirs(newpath3)

  # transpose the csv
  with open(tempath+'temp2.csv') as f, open(newpath + str(i+seizure_no) +'.csv', 'w') as fw:
      writer(fw, delimiter=',').writerows(zip(*reader(f, delimiter=',')))

  # read and drop the first col
  df=pd.read_csv(newpath + str(i+seizure_no) +'.csv', header=None)
  df.drop(columns=0, axis=1,inplace=True)

  # Non Specific
  df.to_csv(newpath +str(i+seizure_no) +'.csv', index=False, header=False)

  # InterSpecific
  if patientno!='23' and patientno!='24' :
    df.to_csv(newpath2 + subfile +'_non-seizure(peri-ictal).csv', index=False, header=False)

  # Intraspecific
  if patientno=='1' or patientno=='21' :
    df.to_csv(newpath3 + subfile +'_non-seizure(peri-ictal).csv', index=False, header=False)